In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

In [2]:
from time import sleep
import json
import pandas as pd
import io
import re
import numpy as np
from tqdm import tqdm
import pandas as pd
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from tqdm import tnrange, tqdm_notebook, tqdm

from sklearn import preprocessing
import matplotlib.pyplot as plt
# from plotly.offline import init_notebook_mode, iplot
# import plotly.graph_objs as go
# init_notebook_mode(connected=True)

In [14]:
tweets_raw_file   = '../Dataset/Twitter/Bitcoin_tweets.csv'

In [15]:
df_raw = pd.read_csv(tweets_raw_file,low_memory=False)
print(df_raw.shape)
df_raw.head(5)

(3680507, 13)


,user_name,user_location,user_description,user_created,user_followers,user_friends,user_favourites,user_verified,date,text,hashtags,source,is_retweet
0,DeSota Wilson,"Atlanta, GA","Biz Consultant, real estate, fintech, startups...",2009-04-26 20:05:09,8534.0,7605,4838,False,2021-02-10 23:59:04,Blue Ridge Bank shares halted by NYSE after #b...,['bitcoin'],Twitter Web App,False
1,CryptoND,NaN,😎 BITCOINLIVE is a Dutch platform aimed at inf...,2019-10-17 20:12:10,6769.0,1532,25483,False,2021-02-10 23:58:48,"😎 Today, that's this #Thursday, we will do a ""...","['Thursday', 'Btc', 'wallet', 'security']",Twitter for Android,False
2,Tdlmatias,"London, England","IM Academy : The best #forex, #SelfEducation, ...",2014-11-10 10:50:37,128.0,332,924,False,2021-02-10 23:54:48,"Guys evening, I have read this article about B...",NaN,Twitter Web App,False
3,Crypto is the future,NaN,I will post a lot of buying signals for BTC tr...,2019-09-28 16:48:12,625.0,129,14,False,2021-02-10 23:54:33,$BTC A big chance in a billion! Price: \487264...,"['Bitcoin', 'FX', 'BTC', 'crypto']",dlvr.it,False
4,Alex Kirchmaier 🇦🇹🇸🇪 #FactsSuperspreader,Europa,Co-founder @RENJERJerky | Forbes 30Under30 | I...,2016-02-03 13:15:55,1249.0,1472,10482,False,2021-02-10 23:54:06,This network is secured by 9 508 nodes as of t...,['BTC'],Twitter Web App,False


In [19]:
!dir

 Volume in drive D is WD Blue 1TB
 Volume Serial Number is D660-6B07

 Directory of D:\Desktop\UCSP\BIG DATA\Trabajo Final\Implementacion\Notebooks

11/27/2022  12:48 PM    <DIR>          .
11/27/2022  12:48 PM    <DIR>          ..
11/27/2022  12:14 PM    <DIR>          .ipynb_checkpoints
11/27/2022  12:48 PM        15,831,196 Bitcoin_tweets_clean.csv
11/27/2022  12:33 PM            12,662 Untitled.ipynb
               2 File(s)     15,843,858 bytes
               3 Dir(s)  101,588,721,664 bytes free


In [20]:
tweets_clean_file = '../Dataset/Twitter/Bitcoin_tweets_clean.csv'

In [21]:
# clean df 
df_raw = df_raw.sort_values(by = 'date')
dd = df_raw.sample(frac=0.01, replace=False, random_state=1)
dd.reset_index(inplace=True)
for i,s in enumerate(tqdm(dd['text'],position=0, leave=True)):
    text = str(dd.loc[i, 'text'])
    text = text.replace("#", "")
    text = re.sub('https?://(?:[-\w.]|(?:%[\da-fA-F]{2}))+', '', text, flags=re.MULTILINE)
    text = re.sub('@\\w+ *', '', text, flags=re.MULTILINE)
    dd.loc[i, 'text'] = text
# f = open(tweets_clean_file, 'a+', encoding='utf-8')
dd.to_csv(tweets_clean_file, header=True, encoding='utf-8',index=False)

100%|██████████████████████████████████████████████████████████████████████████| 36805/36805 [00:22<00:00, 1667.92it/s]


In [22]:
df_clean = pd.read_csv(tweets_clean_file)

In [23]:
analyzer = SentimentIntensityAnalyzer()
compound = []
for i,s in enumerate(tqdm(df_clean['text'],position=0, leave=True)):
    # print(i,s)
    vs = analyzer.polarity_scores(str(s))
    compound.append(vs["compound"])
df_clean["compound"] = compound
df_clean.head(2)

100%|█████████████████████████████████████████████████████████████████████████| 36805/36805 [00:02<00:00, 12744.33it/s]


,index,user_name,user_location,user_description,user_created,user_followers,user_friends,user_favourites,user_verified,date,text,hashtags,source,is_retweet,compound
0,1715327,Crypto Price Postings,NaN,An attempt at creating a bot that will tweet o...,2021-06-14 16:19:03,580.0,143.0,575.0,False,2021-11-12 11:19:49,Bitcoin Price (USD): 63998.36 \nEthereum Price...,"['Bitcoin', 'Dogecoin', 'Ethereum', 'Safemoon'...",TwitterBot-6/15/21,False,0.0000
1,859353,intratio,NaN,We have built the state-of-the-art portfolio o...,2020-05-12 14:29:18,732.0,1.0,3.0,False,2021-08-21 03:49:47,$EVLO /T189cHCL3G Evelo Biosciences Inc Our an...,"['investing', 'bitcoin', 'smallbusiness']",intratio,False,-0.5719


In [24]:
scores = []
for i, s in tqdm(df_clean.iterrows(), total=df_clean.shape[0],position=0, leave=True):
    try:
        scores.append(s["compound"] * ((int(s["user_followers"]))) * ((int(s["user_favourites"])+1)/int(s['user_followers']+1)) *((int(s["is_retweet"])+1)))
    except:
        scores.append(np.nan)
df_clean["score"] = scores
df_clean.head(2)

100%|█████████████████████████████████████████████████████████████████████████| 36805/36805 [00:01<00:00, 33644.51it/s]


,index,user_name,user_location,user_description,user_created,user_followers,user_friends,user_favourites,user_verified,date,text,hashtags,source,is_retweet,compound,score
0,1715327,Crypto Price Postings,NaN,An attempt at creating a bot that will tweet o...,2021-06-14 16:19:03,580.0,143.0,575.0,False,2021-11-12 11:19:49,Bitcoin Price (USD): 63998.36 \nEthereum Price...,"['Bitcoin', 'Dogecoin', 'Ethereum', 'Safemoon'...",TwitterBot-6/15/21,False,0.0000,0.000000
1,859353,intratio,NaN,We have built the state-of-the-art portfolio o...,2020-05-12 14:29:18,732.0,1.0,3.0,False,2021-08-21 03:49:47,$EVLO /T189cHCL3G Evelo Biosciences Inc Our an...,"['investing', 'bitcoin', 'smallbusiness']",intratio,False,-0.5719,-2.284479
